In [ ]:
'''
Steve Winnall 10 Aug 2019
This code is work toward the Udacity Machine Learning NanoDegree Capstone Project
"The Art Thief - Is this a Whiteley?"
Using machine learning to identify a "fake" Brett Whiteley paiting.

Train a CNN classifier from scratch using the images in the training and validation set
corresponding to the Whiteley or non-Whitely painting class
Using Keras with the Adadelta optimizer and MSE loss function

'''

#imports
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint 
from keras import optimizers
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.datasets import load_files       
from keras.utils import np_utils
from glob import glob
from scipy import ndimage            
import matplotlib.pyplot as plt                        
%matplotlib inline

#directory locations for training and validation images
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

# input variables
img_width, img_height = 256, 256 
nb_train_samples = 2000  
nb_validation_samples = 1000  
epochs = 20
batch_size = 16

#ensuring the data is in the correct format "channels_first" or "channels_last"
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#create the CNN model
model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape=input_shape))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128,(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256,(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(rate=0.3))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary() #print out the model

#Use Adadelta optimiser and MSE loss function - this combination prevented overfitting and "stuck" local optimization
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)

model.compile(loss='mean_squared_error',
            optimizer=adadelta,
            metrics=['accuracy'])

# Data augmentation configuration we will use for training
# given the small dataset we will perform translation, shear, flip, and rotation over a reasonable range 
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=0.3,
    vertical_flip=True,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=20)

#callbacks
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

# Data augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

#use flow_from_directory() object to create the input data from images in each directory
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    shuffle=True,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    shuffle=True,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=[checkpointer],
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('CNN_scratch.h5')

print(history.history.keys()) 

plt.figure(1)  
   
# summarize history for accuracy  
   
plt.subplot(311)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('Model accuracy')  
plt.ylabel('Accuracy')  
plt.xlabel('Epoch')  
plt.legend(['train', 'test'], loc='lower right')  
   
# summarize history for loss  
   
plt.subplot(313)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('Model loss')  
plt.ylabel('Loss')
plt.xlabel('Epoch')  
plt.legend(['train', 'test'], loc='upper right')  
plt.show()  


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 16)      2320      
_________________________________________________________________
activation_1 (Activation)    (None, 252, 252, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 16)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 16)      2320      
_________________________________________________________________
activation_2 (Activation)    (None, 124, 124, 16)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 16)        0         
__________